In [1]:
import pyrcn

In [2]:
import os

In [4]:
# os.listdir('/Volumes/Untitled')

['System Volume Information',
 '$RECYCLE.BIN',
 '.Spotlight-V100',
 'Xilinx',
 '.fseventsd',
 'ECG_data',
 'Research Code',
 'IDRad',
 'REDS',
 'Vimeo']

In [56]:
# DATA_DIR = '/Volumes/Untitled/IDRad/idrad'
DATA_DIR = 'idrad'
DEFAULT_FILE = 'train/target5_001.hdf5'

# Preprocessing

In [76]:
def range_doppler(data, chirps=256,
                  samples=256,
                  fft_rangesamples=2 ** 10,
                  fft_dopplersamples=2 ** 8,
                  fs=2.0e6,
                  kf=1171875.0e7,
                  min_range=0.5,
                  max_range=10):
    """
    Computes a range-doppler map for a given number of chirps and samples per chirp.
    :param data: FMCW radar data frame consisting of <chirps>x<samples>
    :param chirps: Number of chirps (Np)
    :param samples: Number of samples (N)
    :param fft_rangesamples: Number of samples for the range fft.
    :param fft_dopplersamples: Number of samples for the doppler fft.
    :param fs: Constant depending on the radar recording parameters.
    :param kf: Constant depending on the radar recording parameters.
    :param min_range: Minimum value to take into account for the range axis in the range-doppler map.
    :param max_range: Maximum value to take into account for the range axis in the range-doppler map.
    :return: Returns a 2D dimensional range-doppler map representing the reflected power over all range-doppler bins.
    """

    data = data.reshape(chirps, samples).T
    # Ignore chirp sequence number
    data = data[1:]
    Ny, Nx = data.shape  # rows (N), columns (Np)

    window = np.hanning(Ny)
    scaled = np.sum(window)
    window2d = np.tile(window, (Nx, 1)).T
    data = data * window2d

    # Calculate Range FFT
    x = np.zeros((fft_rangesamples, Nx))
    start_index = int((fft_rangesamples - Ny) / 2)
    x[start_index:start_index + Ny, :] = data
    X = np.fft.fft(x, fft_rangesamples, 0) / scaled * (2.0 / 2048)
    # Extract positive range bins
    X = X[0:fft_rangesamples // 2, :]
    # Extract range
    _freq = np.arange(fft_rangesamples // 2) / float(fft_rangesamples) * fs
    _range = _freq * 3e8 / (2 * kf)
    min_index = np.argmin(np.abs(_range - min_range))
    max_index = np.argmin(np.abs(_range - max_range))

    X = X[min_index: max_index, :]

    # Calculate Doppler FFT
    Ny, Nx = X.shape
    window = np.hanning(Nx)
    scaled = np.sum(window)
    window2d = np.tile(window, (Ny, 1))
    X = X * window2d

    rd = np.zeros((Ny, fft_dopplersamples), dtype='complex_')
    start_index = int((fft_dopplersamples - Nx) / 2)
    rd[:, start_index:start_index + Nx] = X

    range_doppler = np.fft.fft(rd, fft_dopplersamples, 1) / scaled
    range_doppler = np.fft.fftshift(range_doppler, axes=1)

    return np.abs(range_doppler)

def preprocess_file(fname): 
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        nframes = file['radar'].shape[0]

        # Create datasets
        if not 'microdoppler' in file:
            file.create_dataset("microdoppler", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'microdoppler_thresholded' in file:
            file.create_dataset("microdoppler_thresholded", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'range_doppler' in file:
            file.create_dataset("range_doppler", (nframes, 380, 256), dtype='float32', chunks=True)

        
        x = file['range_doppler'][:10,:,:]
        
        #has not been preprocessed
        if np.all(x==0): 
            print('preprocessing')
        
            # Run over each radar frame
            for i in range(nframes): # only take first 1000 
                rd = range_doppler(file['radar'][i]) 
                rd = 20 * np.log10(rd)

                file['range_doppler'][i] = rd
                file['microdoppler'][i] = rd.sum(axis=0)

                rd -= np.amax(rd)
                rd[rd < -45] = -45
                file['microdoppler_thresholded'][i] = rd.sum(axis=0)

                if not i%100: 
                    print("Finished frame %d of %d." % (i + 1, nframes))
                    
def get_range_doppler(fname): 
    '''returns the range doppler'''
    preprocess_file(fname) 
    range_doppler = 0 
    
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        # d['microdoppler'] = file['microdoppler'][:,:]
        # d['microdoppler_thresholded'] = file['microdoppler_thresholded'][:,:]
        range_doppler = file['range_doppler'][:,:,:].sum(axis=(0,1))
    
    return range_doppler

In [77]:
dataset_files = os.listdir(f'{DATA_DIR}/train')
dataset = [] 
labels = []

for fname in dataset_files: 
    if fname[:2] == '._':
        print(fname)
        continue 
    labels.append(int(fname[6]))
    dataset.append(get_range_doppler(f'train/{fname}'))

dataset = np.array(dataset) 
labels = np.array(labels)

._target2_017.hdf5
._target3_105.hdf5
._target3_113.hdf5
._target1_073.hdf5
._target4_051.hdf5
._target3_129.hdf5
._target4_047.hdf5
._target4_067.hdf5
._target3_109.hdf5
._target1_090.hdf5
._target2_021.hdf5
._target1_086.hdf5
._target3_125.hdf5
._target1_069.hdf5
._target1_087.hdf5
._target3_124.hdf5
._target1_068.hdf5
._target2_020.hdf5
._target1_091.hdf5
._target3_108.hdf5
._target4_066.hdf5
._target4_046.hdf5
._target3_128.hdf5
._target4_050.hdf5
._target1_072.hdf5
._target3_112.hdf5
._target3_104.hdf5
._target2_016.hdf5
._target4_036.hdf5
._target4_061.hdf5
._target3_119.hdf5
._target1_096.hdf5
._target1_079.hdf5
._target2_027.hdf5
._target2_031.hdf5
._target1_080.hdf5
._target3_123.hdf5
._target2_011.hdf5
._target3_103.hdf5
._target3_115.hdf5
._target2_007.hdf5
._target1_075.hdf5
._target4_057.hdf5
._target4_041.hdf5
._target4_040.hdf5
._target5_001.hdf5
._target4_056.hdf5
._target1_074.hdf5
._target2_006.hdf5
._target3_114.hdf5
._target3_102.hdf5
._target2_010.hdf5
._target1_08

In [78]:
print(dataset.shape)

(130, 256)


In [52]:
from os.path import isfile

# Layer Making

In [24]:
from pyrcn.base.blocks import InputToNode, BatchIntrinsicPlasticity, NodeToNode, HebbianNodeToNode

In [39]:
initial_params = {'hidden_layer_size': 125, #1000 in 5C
                  'input_activation': 'identity', #
                  'k_in': 256, # number of inputs 
                  'bias_scaling': 0.0, # usually 0 
                  'spectral_radius' : 0.0, 
                  'reservoir_activation': 'tanh', # 2
                  'leakage': 0.05, #equation 6, 17 frames should be 1 step 
                  'bidirectional': True, #bidirectional 
                  'k_rec': 10,
                  'alpha': 1e-5,
                  'random_state': 1,
                  'requires_sequence': True}

In [26]:
default_rd = get_range_doppler(DEFAULT_FILE)

In [31]:
# Bilayered; i2n -> n2o, second layer has fewer (125) nodes

i2n = InputToNode(hidden_layer_size=500, input_activation="tanh", input_scaling=1.0, bias_scaling=0.1)

layer1 = i2n.fit_transform(default_rd)

In [34]:
from pyrcn.echo_state_network import ESNRegressor

In [41]:
n2o = ESNRegressor(input_to_node = i2n, **initial_params)

In [40]:
n2o.fit(dataset, y)

ESNRegressor(input_to_node=InputToNode(bias_scaling=0.0,
                                       input_activation='identity', k_in=256,
                                       random_state=1),
             node_to_node=NodeToNode(bidirectional=True, k_rec=10, leakage=0.05,
                                     random_state=1, spectral_radius=0.0),
             regressor=IncrementalRegression(), requires_sequence=True)